## 读取数据

前面的部分我们基本上已经把模型训练的流程讲完了，现在我们都是使用`cifar10`里的数据读取函数进行数据读入，接下来我们讲如何导入自己的数据集，这里会用到一个`tensorflow`内置的，非常有用的模块
- [`Queue`](#Queue)
- [`.tfrecord`](#.tfrecord)
- [`tf.data`](#tf.data)

下面我们来讲一讲

## Queue

<img src="https://tensorflow.google.cn/images/AnimatedFileQueues.gif">

`tensorflow`提供一种队列方式进行数据的读取, 我们通过读取图片的例子来看看整体用法

In [1]:
from __future__ import print_function
from __future__ import division
from __future__ import absolute_import

import tensorflow as tf

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


将图片名和标签信息读入

In [2]:
with open('example_data/imgs.txt', 'r') as fid:
    lines = fid.readlines()
    
img_names = ['example_data/%s' % line.strip().split()[0] for line in lines]
img_labels = [line.strip().split()[1] for line in lines]

- **tf.train.slice_input_producer**

将输入按照第0维进行切割(可以有多个输入), 生成一个队列

(注意`num_epochs`参数, 表示使用多少次全样本集, 在这里设置为1, 不设置的话默认使用无限次)
(注意`shuffle`参数, 表示是否在一个全样本集内部打乱顺序, 默认设置为`True`)

In [3]:
data_queue = tf.train.slice_input_producer([img_names, img_labels], shuffle=False, num_epochs=1)

- **解析队列**, 生成具体样本

In [4]:
def read(data_queue):
    filename = data_queue[0]
    label = data_queue[1]
    img_file = tf.read_file(filename)
    img_decoded = tf.image.decode_image(img_file)
    # 这里最好设定输出图片的形状, 否则后面无法进行`batch`操作
    # 比如我们可以`resize`到固定大小
    img_decoded.set_shape((32, 32, 3))
    
    return img_decoded, label

- **读取队列**

In [5]:
img, label = read(data_queue)

In [6]:
print(img.get_shape(), label.get_shape())

(32, 32, 3) ()


In [7]:
sess = tf.InteractiveSession()

InternalError: Failed to create session.

注意, 这里需要初始化局部变量

In [ ]:
sess.run(tf.local_variables_initializer())

- **tf.train.Coordinator**

生成一个管理器, 管理读取线程

In [ ]:
coord = tf.train.Coordinator()

- **tf.train.start_queue_runners**

启动线程

In [ ]:
threads = tf.train.start_queue_runners(sess=sess, coord=coord)

- 运行输出

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
while True:
    try:
        py_img, py_label = sess.run([img, label])
        plt.figure(figsize=(1, 1))
        plt.imshow(py_img)
        plt.title(py_label)
        plt.axis('off')
        plt.show()
    except tf.errors.OutOfRangeError:
        # 当报错越界时, 输出信息, 结束循环
        print('Epoch Limited. Done')
        break
    finally:
        # 停止读取线程
        coord.request_stop()
# 等待线程彻底终止
coord.join(threads)

**注意**: 一个`session`只能开启一个队列, 在这里我们先关闭这个`sess`

In [ ]:
sess.close()

上面大家看到了队列的基本操作, 下面我们再来看一些常用的数据操作

- **slice_input_producer: shuffle=True**

在样本集内部打乱样本

In [ ]:
data_queue = tf.train.slice_input_producer([img_names, img_labels], num_epochs=1)

In [ ]:
img, label = read(data_queue)

with tf.Session() as sess:
    sess.run(tf.local_variables_initializer())
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    while True:
        try:
            py_img, py_label = sess.run([img, label])
            print(py_label)
        except tf.errors.OutOfRangeError:
            # 当报错越界时, 输出信息, 结束循环
            print('Epoch Limited. Done')
            break
        finally:
            # 停止读取线程
            coord.request_stop()
    # 等待线程彻底终止
    coord.join(threads)

- **tf.train.batch**

将`batch_size`个样本打包成一次输出

- **tf.train.shuffle_batch**

对样本进行打乱顺序然后打包

In [ ]:
batch_size = 10

min_after_dequeue = 1000
capacity = min_after_dequeue + 3 * batch_size

data_queue = data_queue = tf.train.slice_input_producer([img_names, img_labels], shuffle=False, num_epochs=10)
img, label = read(data_queue)
# 如果不需要打乱样本, 可以用
#imgs, labels = tf.train.batch([img, label], batch_size)
imgs, labels = tf.train.shuffle_batch([img, label], batch_size, capacity=capacity, min_after_dequeue=min_after_dequeue)

with tf.Session() as sess:
    sess.run(tf.local_variables_initializer())
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    for i in range(5):
        try:
            py_imgs, py_labels = sess.run([imgs, labels])
            print(py_imgs.shape, py_labels)
        except tf.errors.OutOfRangeError:
            # 当报错越界时, 输出信息, 结束循环
            print('Epoch Limited. Done')
            break
        finally:
            # 停止读取线程
            coord.request_stop()
    # 等待线程彻底终止
    coord.join(threads)

关于队列的基础知识就介绍到这, 接下来我们来看看`tensorflow`为了在内部高效化读取数据而定义的一种全新文件格式

## .tfrecord

`.tfreocrd`是`tensorflow`特有的数据存储形式. 在使用的时候, 第一步需要将我们自己的数据转换成`.tfrecord`格式的文件, 在之后我们就可以从相应的`.tfrecord`文件中解码读取. 由于`tensorflow`为`.tfrecord`定制了很多读取函数, 因此它比原生的从硬盘中读取的方式效率高一些.

### 生成`.tfrecord`文件

`.tfrecord`文件包含了`tf.train.Example`协议缓冲区, 我们首先需要定义`writer`用来往文件里写入, 然后将数据转换成特定形式, 再调用`writer`进行写入就完成了
- - -
数据需要转换成**`tf.train.Features()`**的形式, 这是一个字典, 

- `key`值是数据的名字

用来处理不同类型的数据, 比如图片和标签就可以分别存诚`img`, `label`两个部分.

- `value`是`tf.train.Feature()`形式的特征

而我们要做的就是把每个单独的数据转换成这种特征. 

- - -

特征有3种:

- bytes_list 将字符串数据存储在这里
- int64_list 将整型标量(也就是一个数)存储在这里
- float_list 将浮点型标量存储在这里

In [ ]:
tfrecord_fname = './example_data.tfrecord'
writer = tf.python_io.TFRecordWriter(tfrecord_fname)

In [ ]:
import cv2

In [ ]:
for img_name, img_label in zip(img_names, img_labels):
    # 读取图片
    img_raw = cv2.imread(img_name)
    # 将图片数组转换成字符串形式, 后面可以解码
    img_raw = img_raw.tostring()
    
    # 定义一个样本
    example = tf.train.Example(features=tf.train.Features(
        # 定义特征字典
        feature={
            # 将`img_label`作为'img_label'的值存入样本中, 这里它是一个字符串, 所以我们用`bytes_list`
            'img_label': tf.train.Feature(bytes_list = tf.train.BytesList(value=[img_label.encode()])), 
            # 将`img_raw`作为'img_raw'的值存入样本中, 图片已经转换成了字符串, 同理`bytes_list`
            'img_raw': tf.train.Feature(bytes_list = tf.train.BytesList(value=[img_raw]))
        }))
    # 将样本序列化成字符串后写入`.tfrecord`文件中
    writer.write(example.SerializeToString())

In [ ]:
# 关闭读写器
writer.close()

这时候我们就发现在当前目录下多了一个`example_data.tfrecord`的文件.

从上面的过程我们就可以发现, `.tfrecord`可以将一个样本的所有信息整合在一起, 非常方便

### 读取`.tfrecord`文件

现在我们再来看看如何读取`.tfrecord`文件到内存中

- 生成一个文件名队列, 这个队列只有一个元素

In [ ]:
filename_queue = tf.train.string_input_producer(['example_data.tfrecord'], num_epochs=1)

- 定义一个读取器

In [ ]:
reader = tf.TFRecordReader()

- 返回文件名和文件

In [ ]:
_, serialized_example = reader.read(filename_queue)

- 按照指定特征解析`example`里面的内容

In [ ]:
features = tf.parse_single_example(serialized_example, 
                                   features={
                                       'img_label': tf.FixedLenFeature([], tf.string), 
                                       'img_raw': tf.FixedLenFeature([], tf.string)
                                   })

In [ ]:
img = tf.decode_raw(features['img_raw'], tf.uint8)
img = tf.reshape(img, (32, 32, 3))

In [ ]:
label = features['img_label']

In [ ]:
with tf.Session() as sess:
    sess.run(tf.local_variables_initializer())
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    while True:
        try:
            py_img, py_label = sess.run([img, label])
            print(py_img.shape, py_label)
        except tf.errors.OutOfRangeError:
            # 当报错越界时, 输出信息, 结束循环
            print('Epoch Limited. Done')
            break
        finally:
            # 停止读取线程
            coord.request_stop()
    # 等待线程彻底终止
    coord.join(threads)

读取完全正确, 关于`.tfrecord`还有很多内容没有详述, 大家可以参考下面几个链接继续深入学习:
- https://tensorflow.google.cn/versions/r1.2/programmers_guide/reading_data
- http://blog.csdn.net/u010223750/article/details/70482498

接下来为大家介绍`tf-1.3`版本纳入`contrib`中, `tf-1.4`版本正式纳入核心库的`tf.data`模块

## tf.data
> `tf.data`可以帮助我们更轻松地处理超量级, 不同格式, 需要进行复杂变换的数据

使用`tf.data`由两个部分构成:
- 构建一个数据集(`tf.data.Dataset`)
- 从数据集中获取元素(`tf.data.Iterator`)

### 用法
我们先用通过一个`numpy`的一维数组构建和使用`dataset`做为例子来看看整体用法

In [ ]:
import numpy as np

# 构建一个`[0, 5)之间长度为5的数组`
x = np.random.randint(0, 5, size=5)
print(x)

#### 构建数据集
从`x`构建一个`dataset`,它第`i`个元素正是`x`的第`i`个元素

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(x)

#### 查看数据集的信息

In [ ]:
print(dataset.output_types)
print(dataset.output_shapes)

#### 生成一个在数据集上的迭代器

In [ ]:
iterator = dataset.make_one_shot_iterator()

#### 获取数据集中的元素

In [ ]:
next_elm = iterator.get_next()

#### 迭代读取数据集中的元素

In [ ]:
sess = tf.InteractiveSession()

In [ ]:
for i in range(5):
    print(sess.run(next_elm))

**注意**`x`有5个元素, 如果我们跑5次以上迭代的话就会报越界的错误

可以发现, 加入了迭代器机制后读取数据变得非常简单优雅. 下面再介绍关于`dataset`的其他基本用法

#### 多种数据构成的数据集

重新构造迭代器和获取元素的`op`

In [ ]:
def dataset_run(sess, dataset, max_step):
    iterator = dataset.make_one_shot_iterator()
    next_elm= iterator.get_next()

    for i in range(max_step):
        # 如果报越界错误, 打印信息并退出循环
        try:
            print(sess.run(next_elm))
        except tf.errors.OutOfRangeError:
            print('Epoch limited, done')
            break

- **`tf.data.Dataset.zip`**

将两个`dataset`进行拼接

先定义一个`[5, 2]`的数据集, 和一个包含5个字符串的数据集

In [ ]:
dataset1 = tf.data.Dataset.from_tensor_slices(np.random.rand(5, 2))

In [ ]:
dataset_run(sess, dataset1, 5)

In [ ]:
dataset2 = tf.data.Dataset.from_tensor_slices(['one', 'two', 'three', 'four', 'five'])

然后把`dataset1`和`dataset2`通过`tf.data.Dataset.zip`函数连接在一起

In [ ]:
dataset = tf.data.Dataset.zip((dataset1, dataset2))

In [ ]:
print(dataset.output_shapes)
print(dataset.output_types)

In [ ]:
dataset_run(sess, dataset, 5)

- 直接在`tf.data.Dataset.from_tensor_slices`中定义两个数据集

还可以用字典的形式给数据加名字用来区别

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(
    {'a': np.random.rand(5, 2), 
     'b': np.random.randint(0, 2, [5])})

In [ ]:
print(dataset.output_types)
print(dataset.output_shapes)

#### 对数据集进行变换

In [ ]:
# 每个元素+1
def add_one(x):
    return x + 1

- **`dataset.map`**

类似`python`下的`map`函数, `dataset`的`map`函数也有相同的功能

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(np.random.randint(0, 5, [5]))
dataset = dataset.map(lambda x: add_one(x))

In [ ]:
dataset_run(sess, dataset, 10)

- **`dataset.filter`**

类似`python`下的`filter`函数

In [ ]:
# 留下小于3的元素
dataset_filtered = dataset.filter(lambda x: tf.less(x, 3))

In [ ]:
dataset_run(sess, dataset_filtered, 10)

- **`dataset.flat_map`**

和`dataset.map`功能前面功能相同, 后面会把结果展开成一个向量

- **`dataset.repeat`**

上面的操作只能读取一次样本集, `repeat`函数能够帮助我们任意次读取样本集

In [ ]:
dataset = dataset.repeat()

In [ ]:
dataset_run(sess, dataset, 15)

如果你想要读取`n`次样本集, 使用

`dataset = dataset.repeat(n)`

- **`dataset.shuffle`**

打乱数据集样本

In [ ]:
dataset = dataset.shuffle(100)

In [ ]:
dataset_run(sess, dataset, 10)

- **`dataset.batch`**

一次读取`batch_size`个样本

In [ ]:
dataset = dataset.batch(5)

In [ ]:
dataset_run(sess, dataset, 5)

上面介绍的这些`tf.data`的基本方法可以满足我们大部分时候的需求了, 我们再用读取图片数据为例子作为本章的结束

获取图片文件名和标签名列表

In [ ]:
with open('example_data/imgs.txt') as fid:
    lines = fid.readlines()
    
filenames = ['example_data/%s' % line.strip().split()[0] for line in lines]
labels = [line.strip().split()[1] for line in lines]

独立构造图片数据集以及标签数据集

In [ ]:
image_dataset = tf.data.Dataset.from_tensor_slices(filenames)
label_dataset = tf.data.Dataset.from_tensor_slices(labels)

在这里`image_dataset`的元素是一个字符串, 我们需要将它转化成图片本身

In [ ]:
def read_img(name):
    img_file = tf.read_file(name)
    img_decoded = tf.image.decode_image(img_file, channels=3)
    return img_decoded

In [ ]:
image_dataset = image_dataset.map(lambda name: read_img(name))

我们还可以对图片进行变换, 也就是预处理

In [ ]:
def distort_img(img):
    img_flip_lr = tf.image.random_flip_left_right(img)
    img_flip_ud = tf.image.random_flip_up_down(img_flip_lr)
    img_adj_bri = tf.image.random_brightness(img_flip_ud, 0.5)
    img_adj_con = tf.image.random_contrast(img_adj_bri, 0.5, 1)
    
    return img_adj_con

In [ ]:
image_dataset = image_dataset.map(lambda img: distort_img(img))

现在将两部分数据融合

In [ ]:
dataset = tf.data.Dataset.zip((image_dataset, label_dataset))

In [ ]:
# 设定循环样本集10次
dataset = dataset.repeat(10)

In [ ]:
# 打乱样本
dataset = dataset.shuffle(100)

In [ ]:
# 设定`batch_size`
dataset = dataset.batch(5)

现在数据集以及处理完成, 我们来看看实际效果

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def dataset_visualize(sess, dataset, max_step):
    iterator = dataset.make_one_shot_iterator()
    images, labels = iterator.get_next()

    for i in range(max_step):
        try:
            np_imgs, np_labels = sess.run([images, labels])
            _, axes = plt.subplots(1, 5, figsize=(8, 8))
            for n in range(5):
                axes[n].imshow(np_imgs[n])
                axes[n].set_title(np_labels[n])
                axes[n].axes.get_xaxis().set_visible(False)
                axes[n].axes.get_yaxis().set_visible(False)
            plt.show()
        except tf.errors.OutOfRangeError:
            print('Epoch limited, done')
            break

In [ ]:
dataset_visualize(sess, dataset, 5)

In [ ]:
sess.close()

## 结语

我们学习了如何使用`tensorflow`的各种方法方便地进行数据的读取和处理, 下面我们再看看如何在自己的数据集上训练以及一些技巧